# GSPO Llama3.1-8B Demo

This notebook demonstrates GSPO (Group Sequence Policy Optimization) training using the unified `rl_train` entry point. GSPO shares the same infrastructure as GRPO but applies a sequence-aware clipping objective. Use this notebook when you want to experiment with the `loss_algo=gspo-token` mode for reasoning workloads.

## Prerequisites

- Access to a TPU VM with sufficient memory for Llama3.1-8B or a multi-host Pathways cluster
- A scanned checkpoint compatible with MaxText
- A valid Hugging Face access token if the tokenizer/model pull requires authentication


## Setup

Install dependencies and set up the environment:


In [ ]:
# Clone MaxText repository
!git clone https://github.com/AI-Hypercomputer/maxtext.git
%cd maxtext


In [ ]:
# Install dependencies
!chmod +x setup.sh
!./setup.sh

# Install GRPO/GSPO-specific dependencies
!./src/MaxText/examples/install_tunix_vllm_requirement.sh

# Install additional requirements
%pip install --force-reinstall numpy==2.1.2
%pip install nest_asyncio

import nest_asyncio
nest_asyncio.apply()  # Fix for Colab event loop


## Configuration

Set up the training parameters. Defaults target Llama3.1-8B with GSPO loss:


### Multi-host Pathways

To use Pathways with GSPO:
- Confirm `use_pathways=True` inside `rl.yml` (enabled by default).
- Override `trainer_devices_fraction` and `sampler_devices_fraction` in `config_argv` to partition the mesh across hosts.
- Export Pathways runtime variables such as `JAX_PLATFORMS=proxy`, `ENABLE_PATHWAYS_PERSISTENCE=1`, and `JAX_BACKEND_TARGET=grpc://<controller-ip>:29000` before invoking the training cell.
- Adjust `chips_per_vm` to match your TPU slice topology so trainer and rollout workers shard correctly.


In [ ]:
# Configuration for GSPO training
import os

# Set up paths (adjust if needed)
MAXTEXT_REPO_ROOT = os.path.expanduser("~") + "/maxtext"

# Hardcoded defaults for Llama3.1-8B GSPO
MODEL_NAME = "llama3.1-8b"
HF_REPO_ID = "meta-llama/Llama-3.1-8B-Instruct"
CHAT_TEMPLATE_PATH = "src/MaxText/examples/chat_templates/gsm8k_rl.json"
LOSS_ALGO = "gspo-token"

# Required: Set these before running
MODEL_CHECKPOINT_PATH = "gs://maxtext-model-checkpoints/llama3.1-8b/2025-01-23-19-04/scanned/0/items"  # Update!
OUTPUT_DIRECTORY = "/tmp/gspo_output"  # Update!
HF_TOKEN = os.environ.get("HF_TOKEN", "")  # Set HF_TOKEN environment variable

# Optional: Override training parameters
STEPS = 10  # Reduced for demo purposes
PER_DEVICE_BATCH_SIZE = 1
LEARNING_RATE = 3e-6
NUM_GENERATIONS = 2
GSPO_BETA = 0.08
GSPO_EPSILON = 0.2
CHIPS_PER_VM = 4

print(f"📁 MaxText Home: {MAXTEXT_REPO_ROOT}")
print(f"🤖 Model: {MODEL_NAME}")
print(f"📦 Checkpoint: {MODEL_CHECKPOINT_PATH}")
print(f"💾 Output: {OUTPUT_DIRECTORY}")
print(f"🔑 HF Token: {'✅ Set' if HF_TOKEN else '❌ Missing - set HF_TOKEN env var'}")
print(f"📊 Steps: {STEPS}")
print(f"🎯 Loss Algorithm: {LOSS_ALGO}")


In [ ]:
# Import required modules
import os
import sys
from pathlib import Path

# Add MaxText to Python path
maxtext_path = Path(MAXTEXT_REPO_ROOT) / "src" / "MaxText"
sys.path.insert(0, str(maxtext_path))

from MaxText import pyconfig, max_utils
from MaxText.rl.train_rl import rl_train
import jax

# Initialize JAX and Pathways
import pathwaysutils
pathwaysutils.initialize()
jax.config.update("jax_default_prng_impl", "unsafe_rbg")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
os.environ["SKIP_JAX_PRECOMPILE"] = "1"  # Faster startup for vLLM

if "xla_tpu_spmd_rng_bit_generator_unsafe" not in os.environ.get("LIBTPU_INIT_ARGS", ""):
    os.environ["LIBTPU_INIT_ARGS"] = (
        os.environ.get("LIBTPU_INIT_ARGS", "") + " --xla_tpu_spmd_rng_bit_generator_unsafe=true"
    )

print("✅ Successfully imported modules")
print(f"📁 MaxText path: {maxtext_path}")


In [ ]:
# Build configuration for GSPO training
config_file = os.path.join(MAXTEXT_REPO_ROOT, "src/MaxText/configs/rl.yml")

# Verify chat template exists
if not os.path.exists(os.path.join(MAXTEXT_REPO_ROOT, CHAT_TEMPLATE_PATH)):
    raise FileNotFoundError(f"Chat template not found: {CHAT_TEMPLATE_PATH}")

config_argv = [
    "",  # argv[0] placeholder
    config_file,
    f"model_name={MODEL_NAME}",
    f"tokenizer_path={HF_REPO_ID}",
    f"hf_model_name={HF_REPO_ID}",
    f"chat_template_path={CHAT_TEMPLATE_PATH}",
    f"load_parameters_path={MODEL_CHECKPOINT_PATH}",
    f"base_output_directory={OUTPUT_DIRECTORY}",
    f"hf_access_token={HF_TOKEN}",
    f"steps={STEPS}",
    f"per_device_batch_size={PER_DEVICE_BATCH_SIZE}",
    f"learning_rate={LEARNING_RATE}",
    f"num_generations={NUM_GENERATIONS}",
    f"grpo_beta={GSPO_BETA}",
    f"grpo_epsilon={GSPO_EPSILON}",
    f"chips_per_vm={CHIPS_PER_VM}",
    f"loss_algo={LOSS_ALGO}",
]

print(f"🔧 Initializing configuration from: {config_file}")
config = pyconfig.initialize(config_argv)
max_utils.print_system_information()

print("\n✅ Configuration initialized successfully")
print(f"📊 Training steps: {config.steps}")
print(f"📁 Output directory: {config.base_output_directory}")
print(f"🤖 Model: {config.model_name}")
print(f"🎯 Loss algorithm: {config.loss_algo}")


In [ ]:
# Execute GSPO training
print("\n" + "="*80)
print("🚀 Starting GSPO Training...")
print("="*80)

try:
    rl_train(config)

    print("\n" + "="*80)
    print("✅ GSPO Training Completed Successfully!")
    print("="*80)
    print(f"📁 Checkpoints saved to: {config.checkpoint_dir}")
    print(f"📊 TensorBoard logs: {config.tensorboard_dir}")
    print(f"🎯 Model ready for inference!")

except Exception as e:
    print("\n" + "="*80)
    print("❌ GSPO Training Failed!")
    print("="*80)
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()
    print("\n💡 Common issues:")
    print("  - Check that MODEL_CHECKPOINT_PATH points to a valid checkpoint")
    print("  - Ensure HF_TOKEN environment variable is set")
    print("  - Verify OUTPUT_DIRECTORY is writable")
    print("  - Check hardware requirements (TPU/GPU availability)")


## 📚 Learn More

- **CLI Usage**: `python3 -m src.MaxText.rl.train_rl src/MaxText/configs/rl.yml --model_name=llama3.1-8b loss_algo=gspo-token ...`
- **Configuration Reference**: `src/MaxText/configs/rl.yml`
- **Implementation**: `src/MaxText/rl/train_rl.py` (search for `loss_algo` handling)
- **Related Notebook**: `src/MaxText/examples/grpo_llama3_1_8b_demo.ipynb` for GRPO comparison
